In [22]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Utils.utils import create_URM, create_ICM, combine_matrices
from src.Hybrids.MergingModelsByScores import MergeThreeModelsByScores
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from skopt.space import Real,Categorical
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

In [21]:
##CREATE URM,ICM, ICM STACKED COMPLETELY TO TRAIN THE SINGLE RECOMMENDERS, AND STACKED PARTIALLY TO TUNE THE HYBRID
URM = create_URM()
ICM = create_ICM()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
combined_matrices = combine_matrices(ICM=ICM, URM=URM_train)
ICM_combined = combine_matrices(ICM=ICM,URM=URM)

EvaluatorHoldout: Ignoring 13642 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13630 ( 0.1%) Users that have less than 1 test interactions


In [7]:
### P3ALPHA Recommender
P3Alpha_recommender = P3alphaRecommender(URM_train=ICM_combined)
P3Alpha_recommender.fit(topK=64, alpha=0.7424811053766465, implicit=True, normalize_similarity=False)

In [8]:
## IALS Recommender
IALS_recommender = IALSRecommender(URM_train=ICM_combined)
IALS_recommender.fit(epochs=50, num_factors=40, alpha=0.7917156202136513, reg=1.0)

IALSRecommenderLinear: Epoch 1 of 50. Elapsed time 6.46 sec
IALSRecommenderLinear: Epoch 2 of 50. Elapsed time 12.63 sec
IALSRecommenderLinear: Epoch 3 of 50. Elapsed time 18.92 sec
IALSRecommenderLinear: Epoch 4 of 50. Elapsed time 25.04 sec
IALSRecommenderLinear: Epoch 5 of 50. Elapsed time 31.19 sec
IALSRecommenderLinear: Epoch 6 of 50. Elapsed time 37.34 sec
IALSRecommenderLinear: Epoch 7 of 50. Elapsed time 43.57 sec
IALSRecommenderLinear: Epoch 8 of 50. Elapsed time 49.81 sec
IALSRecommenderLinear: Epoch 9 of 50. Elapsed time 56.07 sec
IALSRecommenderLinear: Epoch 10 of 50. Elapsed time 1.04 min
IALSRecommenderLinear: Epoch 11 of 50. Elapsed time 1.14 min
IALSRecommenderLinear: Epoch 12 of 50. Elapsed time 1.24 min
IALSRecommenderLinear: Epoch 13 of 50. Elapsed time 1.35 min
IALSRecommenderLinear: Epoch 14 of 50. Elapsed time 1.45 min
IALSRecommenderLinear: Epoch 15 of 50. Elapsed time 1.55 min
IALSRecommenderLinear: Epoch 16 of 50. Elapsed time 1.66 min
IALSRecommenderLinear: Ep

In [9]:
##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=ICM_combined)
SLIME_recommender.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)

100%|████████████████████████████████████▉| 18032/18059 [11:17<00:00, 52.46it/s]

In [23]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

hyperparameters_range_dictionary = {
    "alpha": Real(0.0, 1.0),
    "beta": Real(0.0, 1.0),
    "gamma": Real(0.0, 1.0)
}

recommenders = [P3Alpha_recommender, IALS_recommender, SLIME_recommender]

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[combined_matrices, recommenders],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=MergeThreeModelsByScores,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 200
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=MergeThreeModelsByScores.RECOMMENDER_NAME,
                    save_model="best"
                    )



Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'alpha': 0.07009833257309385, 'beta': 0.03819256091554402, 'gamma': 0.33607859092887277}
EvaluatorHoldout: Processed 13630 (100.0%) in 14.08 sec. Users per second: 968
SearchBayesianSkopt: New best config found. Config 0: {'alpha': 0.07009833257309385, 'beta': 0.03819256091554402, 'gamma': 0.33607859092887277} - results: PRECISION: 0.2242700, PRECISION_RECALL_MIN_DEN: 0.2277000, RECALL: 0.0638620, MAP: 0.1039026, MAP_MIN_DEN: 0.1053597, MRR: 0.4181890, NDCG: 0.2269106, F1: 0.0994151, HIT_RATE: 0.9017608, ARHR_ALL_HITS: 0.6586543, NOVELTY: 0.0053441, AVERAGE_POPULARITY: 0.6455894, DIVERSITY_MEAN_INTER_LIST: 0.8212558, DIVERSITY_HERFINDAHL: 0.9821196, COVERAGE_ITEM: 0.0651199, COVERAGE_ITEM_CORRECT: 0.0317293, COVERAGE_USER: 0.9985348, COVERAGE_USER_CORRECT: 0.9004396, DIVERSITY_GINI: 0.0047543, SHANNON_ENTROPY: 6.6200172, RATIO_DIVERSITY_HERFINDAHL: 0.9824888, RATIO_DIVERSITY_GINI: 0.0183

100%|████████████████████████████████████▉| 18056/18059 [56:27<00:00,  5.33it/s]


EvaluatorHoldout: Processed 13630 (100.0%) in 13.66 sec. Users per second: 998
SearchBayesianSkopt: Config 4 is suboptimal. Config: {'alpha': 0.3526800189571952, 'beta': 0.10634681618984146, 'gamma': 0.09308834643923493} - results: PRECISION: 0.1955979, PRECISION_RECALL_MIN_DEN: 0.1985134, RECALL: 0.0541983, MAP: 0.0895061, MAP_MIN_DEN: 0.0907237, MRR: 0.3987673, NDCG: 0.2014695, F1: 0.0848778, HIT_RATE: 0.8609685, ARHR_ALL_HITS: 0.5958265, NOVELTY: 0.0052281, AVERAGE_POPULARITY: 0.7037438, DIVERSITY_MEAN_INTER_LIST: 0.7426618, DIVERSITY_HERFINDAHL: 0.9742607, COVERAGE_ITEM: 0.0223711, COVERAGE_ITEM_CORRECT: 0.0142865, COVERAGE_USER: 0.9985348, COVERAGE_USER_CORRECT: 0.8597070, DIVERSITY_GINI: 0.0023020, SHANNON_ENTROPY: 5.7492816, RATIO_DIVERSITY_HERFINDAHL: 0.9746270, RATIO_DIVERSITY_GINI: 0.0088785, RATIO_SHANNON_ENTROPY: 0.4620154, RATIO_AVERAGE_POPULARITY: 3.5380960, RATIO_NOVELTY: 0.0273491, 

Iteration No: 5 ended. Evaluation done at random point.
Time taken: 13.7759
Function va

EvaluatorHoldout: Processed 13630 (100.0%) in 13.58 sec. Users per second: 1003
SearchBayesianSkopt: Config 10 is suboptimal. Config: {'alpha': 0.9708698134116169, 'beta': 0.551842018898332, 'gamma': 0.8304175189283375} - results: PRECISION: 0.2040279, PRECISION_RECALL_MIN_DEN: 0.2072610, RECALL: 0.0574570, MAP: 0.0936101, MAP_MIN_DEN: 0.0948192, MRR: 0.4049461, NDCG: 0.2090069, F1: 0.0896636, HIT_RATE: 0.8775495, ARHR_ALL_HITS: 0.6144476, NOVELTY: 0.0052496, AVERAGE_POPULARITY: 0.6924545, DIVERSITY_MEAN_INTER_LIST: 0.7612292, DIVERSITY_HERFINDAHL: 0.9761173, COVERAGE_ITEM: 0.0305111, COVERAGE_ITEM_CORRECT: 0.0163907, COVERAGE_USER: 0.9985348, COVERAGE_USER_CORRECT: 0.8762637, DIVERSITY_GINI: 0.0026515, SHANNON_ENTROPY: 5.9323054, RATIO_DIVERSITY_HERFINDAHL: 0.9764843, RATIO_DIVERSITY_GINI: 0.0102264, RATIO_SHANNON_ENTROPY: 0.4767233, RATIO_AVERAGE_POPULARITY: 3.4813387, RATIO_NOVELTY: 0.0274615, 

Iteration No: 11 ended. Evaluation done at random point.
Time taken: 13.7427
Function va

EvaluatorHoldout: Processed 13630 (100.0%) in 13.47 sec. Users per second: 1012
SearchBayesianSkopt: Config 16 is suboptimal. Config: {'alpha': 0.4096456692929219, 'beta': 0.43014390020437643, 'gamma': 0.9181835824051782} - results: PRECISION: 0.2168085, PRECISION_RECALL_MIN_DEN: 0.2202490, RECALL: 0.0616617, MAP: 0.0997988, MAP_MIN_DEN: 0.1012027, MRR: 0.4124916, NDCG: 0.2200932, F1: 0.0960159, HIT_RATE: 0.8936904, ARHR_ALL_HITS: 0.6408831, NOVELTY: 0.0052930, AVERAGE_POPULARITY: 0.6700830, DIVERSITY_MEAN_INTER_LIST: 0.7932203, DIVERSITY_HERFINDAHL: 0.9793162, COVERAGE_ITEM: 0.0459605, COVERAGE_ITEM_CORRECT: 0.0219281, COVERAGE_USER: 0.9985348, COVERAGE_USER_CORRECT: 0.8923810, DIVERSITY_GINI: 0.0034977, SHANNON_ENTROPY: 6.2720411, RATIO_DIVERSITY_HERFINDAHL: 0.9796844, RATIO_DIVERSITY_GINI: 0.0134903, RATIO_SHANNON_ENTROPY: 0.5040247, RATIO_AVERAGE_POPULARITY: 3.3688650, RATIO_NOVELTY: 0.0276885, 

Iteration No: 17 ended. Evaluation done at random point.
Time taken: 13.5893
Function 

...

Iteration No: 172 ended. Search finished for the next optimal point.
Time taken: 16.0434
Function value obtained: -0.1212
Current minimum: -0.1213
Iteration No: 173 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.0, 'beta': 0.39964410414965484, 'gamma': 0.7436149345635459}
EvaluatorHoldout: Processed 13630 (100.0%) in 13.92 sec. Users per second: 979
SearchBayesianSkopt: Config 172 is suboptimal. Config: {'alpha': 0.0, 'beta': 0.39964410414965484, 'gamma': 0.7436149345635459} - results: PRECISION: 0.2486134, PRECISION_RECALL_MIN_DEN: 0.2526356, RECALL: 0.0712602, MAP: 0.1212590, MAP_MIN_DEN: 0.1229729, MRR: 0.4647607, NDCG: 0.2546474, F1: 0.1107703, HIT_RATE: 0.9259721, ARHR_ALL_HITS: 0.7481773, NOVELTY: 0.0056051, AVERAGE_POPULARITY: 0.5063123, DIVERSITY_MEAN_INTER_LIST: 0.9301702, DIVERSITY_HERFINDAHL: 0.9930102, COVERAGE_ITEM: 0.0890415, COVERAGE_ITEM_CORRECT: 0.0576444, COVERAGE_USER: 0.9985348, COVERAGE_USER_CORRECT: 0.9246154, DIVER

Iteration No: 179 ended. Search finished for the next optimal point.
Time taken: 16.4310
Function value obtained: -0.1047
Current minimum: -0.1213
Iteration No: 180 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.0, 'beta': 0.45269407840828174, 'gamma': 0.8762590213395784}
EvaluatorHoldout: Processed 13630 (100.0%) in 14.05 sec. Users per second: 970
SearchBayesianSkopt: Config 179 is suboptimal. Config: {'alpha': 0.0, 'beta': 0.45269407840828174, 'gamma': 0.8762590213395784} - results: PRECISION: 0.2486280, PRECISION_RECALL_MIN_DEN: 0.2526508, RECALL: 0.0712385, MAP: 0.1212682, MAP_MIN_DEN: 0.1229804, MRR: 0.4645799, NDCG: 0.2546294, F1: 0.1107455, HIT_RATE: 0.9254585, ARHR_ALL_HITS: 0.7480349, NOVELTY: 0.0056063, AVERAGE_POPULARITY: 0.5059721, DIVERSITY_MEAN_INTER_LIST: 0.9302544, DIVERSITY_HERFINDAHL: 0.9930186, COVERAGE_ITEM: 0.0893737, COVERAGE_ITEM_CORRECT: 0.0578105, COVERAGE_USER: 0.9985348, COVERAGE_USER_CORRECT: 0.9241026, DIVER

Iteration No: 186 ended. Search finished for the next optimal point.
Time taken: 16.4262
Function value obtained: -0.1212
Current minimum: -0.1213
Iteration No: 187 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.0, 'beta': 0.47883189989148695, 'gamma': 0.889524063793863}
EvaluatorHoldout: Processed 13630 (100.0%) in 13.84 sec. Users per second: 985
SearchBayesianSkopt: Config 186 is suboptimal. Config: {'alpha': 0.0, 'beta': 0.47883189989148695, 'gamma': 0.889524063793863} - results: PRECISION: 0.2485913, PRECISION_RECALL_MIN_DEN: 0.2526136, RECALL: 0.0712556, MAP: 0.1212552, MAP_MIN_DEN: 0.1229691, MRR: 0.4647510, NDCG: 0.2546312, F1: 0.1107625, HIT_RATE: 0.9258988, ARHR_ALL_HITS: 0.7481493, NOVELTY: 0.0056051, AVERAGE_POPULARITY: 0.5063257, DIVERSITY_MEAN_INTER_LIST: 0.9301701, DIVERSITY_HERFINDAHL: 0.9930102, COVERAGE_ITEM: 0.0889307, COVERAGE_ITEM_CORRECT: 0.0576444, COVERAGE_USER: 0.9985348, COVERAGE_USER_CORRECT: 0.9245421, DIVERSI

Iteration No: 193 ended. Search finished for the next optimal point.
Time taken: 16.8539
Function value obtained: -0.1213
Current minimum: -0.1213
Iteration No: 194 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'alpha': 0.7784633173905555, 'beta': 0.9983173872658038, 'gamma': 0.5542688426930815}
EvaluatorHoldout: Processed 13630 (100.0%) in 13.80 sec. Users per second: 988
SearchBayesianSkopt: Config 193 is suboptimal. Config: {'alpha': 0.7784633173905555, 'beta': 0.9983173872658038, 'gamma': 0.5542688426930815} - results: PRECISION: 0.2079897, PRECISION_RECALL_MIN_DEN: 0.2114162, RECALL: 0.0588987, MAP: 0.0960725, MAP_MIN_DEN: 0.0973877, MRR: 0.4111967, NDCG: 0.2133225, F1: 0.0918011, HIT_RATE: 0.8829787, ARHR_ALL_HITS: 0.6273120, NOVELTY: 0.0052602, AVERAGE_POPULARITY: 0.6862171, DIVERSITY_MEAN_INTER_LIST: 0.7715009, DIVERSITY_HERFINDAHL: 0.9771444, COVERAGE_ITEM: 0.0292929, COVERAGE_ITEM_CORRECT: 0.0176089, COVERAGE_USER: 0.9985348, COVERAGE_US

Iteration No: 200 ended. Search finished for the next optimal point.
Time taken: 16.9147
Function value obtained: -0.0879
Current minimum: -0.1213
SearchBayesianSkopt: Search complete. Best config is 111: {'alpha': 0.0, 'beta': 0.5157517801733741, 'gamma': 0.9201857532665333}



In [34]:
recommender = MergeThreeModelsByScores(ICM_combined,recommenders)
recommender.fit(alpha=0.0, beta=0.3023745211799622, gamma=1.0)

In [35]:
result_df, _ = evaluator_validation.evaluateRecommender(recommender)
print(result_df.loc[10])

EvaluatorHoldout: Processed 13630 (100.0%) in 13.79 sec. Users per second: 988
PRECISION                          0.0
PRECISION_RECALL_MIN_DEN           0.0
RECALL                             0.0
MAP                                0.0
MAP_MIN_DEN                        0.0
MRR                                0.0
NDCG                               0.0
F1                                 0.0
HIT_RATE                           0.0
ARHR_ALL_HITS                      0.0
NOVELTY                       0.005669
AVERAGE_POPULARITY            0.476534
DIVERSITY_MEAN_INTER_LIST     0.949502
DIVERSITY_HERFINDAHL          0.994943
COVERAGE_ITEM                 0.111523
COVERAGE_ITEM_CORRECT              0.0
COVERAGE_USER                 0.998535
COVERAGE_USER_CORRECT              0.0
DIVERSITY_GINI                0.017997
SHANNON_ENTROPY                8.61123
RATIO_DIVERSITY_HERFINDAHL     0.99532
RATIO_DIVERSITY_GINI          0.070205
RATIO_SHANNON_ENTROPY         0.692777
RATIO_AVERAGE_POPULARITY

In [38]:
recommender_new = MergeThreeModelsByScores(ICM_combined,recommenders)
recommender_new.fit(alpha=0.0, beta=0.5157517801733741, gamma=0.9201857532665333)

In [39]:
result_df, _ = evaluator_validation.evaluateRecommender(recommender_new)
print(result_df.loc[10])

EvaluatorHoldout: Processed 13630 (100.0%) in 13.37 sec. Users per second: 1019
PRECISION                          0.0
PRECISION_RECALL_MIN_DEN           0.0
RECALL                             0.0
MAP                                0.0
MAP_MIN_DEN                        0.0
MRR                                0.0
NDCG                               0.0
F1                                 0.0
HIT_RATE                           0.0
ARHR_ALL_HITS                      0.0
NOVELTY                        0.00565
AVERAGE_POPULARITY             0.48206
DIVERSITY_MEAN_INTER_LIST     0.948083
DIVERSITY_HERFINDAHL          0.994801
COVERAGE_ITEM                 0.101113
COVERAGE_ITEM_CORRECT              0.0
COVERAGE_USER                 0.998535
COVERAGE_USER_CORRECT              0.0
DIVERSITY_GINI                 0.01695
SHANNON_ENTROPY                8.54208
RATIO_DIVERSITY_HERFINDAHL    0.995178
RATIO_DIVERSITY_GINI           0.06612
RATIO_SHANNON_ENTROPY         0.687214
RATIO_AVERAGE_POPULARIT

In [5]:
from Utils.utils import create_submission, write_submission

submission = create_submission(recommender)
write_submission(submission,"SLIME_RP3_IALS_Stacked_hybridBeta")